In [1]:
import os
os.chdir("..")
import descriptor.three_desc_model as tdm
import numpy as np
import pandas as pd
from cutoffs import Polynomial
from scipy.optimize import minimize, differential_evolution
import matplotlib.pyplot as plt
import GPy
from IPython.display import display
#global dataset
dataset = pd.read_csv('hornsrev_data.csv')#, index_col=0)
symbol="Siemens"
numsims=int(2640/80)
turb = [symbol]
nl=tdm.NeighborlistCalculator(cutoff=4001,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)


In [2]:
dataset

,X_coord,Y_coord,Ref_Wind,Num_Turb,Sim_ID,Turbine,Wind_Direction,X_rotated,Y_rotated
0,0.0,0.0,6.112813,80,hornsrev_254_8,T1,-16,0.000000,0.000000
1,59.0,-558.0,6.112391,80,hornsrev_254_8,T2,-16,-97.091204,-552.646630
2,118.0,-1116.0,6.111544,80,hornsrev_254_8,T3,-16,-194.182409,-1105.293261
3,177.0,-1674.0,6.103910,80,hornsrev_254_8,T4,-16,-291.273613,-1657.939891
4,235.0,-2231.0,6.106692,80,hornsrev_254_8,T5,-16,-389.050442,-2209.349622
...,...,...,...,...,...,...,...,...,...
2635,5219.0,-1674.0,5.161035,80,hornsrev_286_8,T94,16,5478.241725,-170.600719
2636,5278.0,-2231.0,5.151867,80,hornsrev_286_8,T95,16,5688.486172,-689.760880
2637,5338.0,-2789.0,5.178098,80,hornsrev_286_8,T96,16,5899.967518,-1209.606665
2638,5397.0,-3347.0,5.149983,80,hornsrev_286_8,T97,16,6110.487603,-1729.728087


In [4]:
def f(x):
    Gs = {"Siemens": [{"type":"G2", "turbine":"Siemens","eta":x[0], "offset": x[3]},
                  {"type":"G4", "elements":"Siemens","eta":x[1], "gamma":x[4]},
                  {"type":"G6", "elements":"Siemens","eta":x[2], "gamma": x[5]}
                 ]}
    #global dataset
    finpr=tdm.FingerprintCalculator(cutoff=4001,Gs=Gs,Rct=3000,delta_R=100,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)
    count=0
    dataset_fp=np.empty(shape=(0, 3))
    dataset_ws=np.empty(shape=(0, 1))
    dataset_pos=np.empty(shape=(0, 2))
    for i in range(numsims):
        numturb=dataset["Num_Turb"].iloc[count]
        position = np.empty((numturb,2))
        ws = np.empty((numturb,1))
        #fp = np.empty((numturb,3))
        for k in range(numturb):
            position[k,0]=dataset.at[count,"X_rotated"]
            position[k,1]=dataset.at[count,"Y_rotated"]
            ws[k,0]=dataset.at[count,"Ref_Wind"]
            count = count+ 1
        neigh=nl.calculate(turb*numturb,position)
        fingerprints=finpr.calculate(turb*numturb,position,neigh,symbol)
        fingerprints=np.array(fingerprints)
        dataset_fp=np.append(dataset_fp,fingerprints,axis=0)
        dataset_ws=np.append(dataset_ws,ws,axis=0)
        dataset_pos=np.append(dataset_pos,position,axis=0)
    dataset_df=np.concatenate((dataset_fp, dataset_ws),axis=1)
    dataset_df = pd.DataFrame(dataset_df, columns = ['Fingerprint(G2)','Fingerprint(G4)','Fingerprint(G6)','Ref_Wind_Speed'])
    #X=dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)"]].to_numpy()
    #Y=dataset[["Ref_Wind_Speed"]].to_numpy()
    train_dataset = dataset_df.sample(frac=0.8, random_state=0)
    test_dataset = dataset_df.drop(train_dataset.index)
    Xtrain=train_dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)"]].to_numpy()
    Ytrain=train_dataset[["Ref_Wind_Speed"]].to_numpy()
    Xtest=test_dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)"]].to_numpy()
    Ytest=test_dataset[["Ref_Wind_Speed"]].to_numpy()
    # define kernel
    ker = GPy.kern.RBF(3,lengthscale=0.1)# + GPy.kern.White(2)
    # create simple GP model
    m = GPy.models.GPRegression(Xtrain,Ytrain,ker)
    # optimize and plot
    m.optimize(max_f_eval = 1000)

    [Ypred,Yvar]=m.predict(Xtest)
    return np.max(Ypred-Ytest)

In [4]:
x0 = [4.25387599, 2.56450515, 2.33043463, 1.0151402, 8.04475192, 0.50753377]
res = minimize(f, x0=x0, method='Nelder-Mead')

[[2.17362992e-01 1.08064160e-03 2.01542618e-04]
 [3.11710296e-01 5.01979964e-02 1.66910964e-01]
 [1.89300807e-01 1.55373910e-01 4.58571618e-07]
 ...
 [7.23135484e-01 3.24098884e-01 2.61404781e-01]
 [2.67474412e-01 3.47269919e-01 2.20160925e-02]
 [9.86847780e-01 9.96826097e-01 1.00000000e+00]]
[[2.40413924e-01 1.08064160e-03 2.01542618e-04]
 [3.31110912e-01 5.01979964e-02 1.66910964e-01]
 [2.13715501e-01 1.55373910e-01 4.58571618e-07]
 ...
 [7.49124939e-01 3.24098884e-01 2.61404781e-01]
 [2.88404409e-01 3.47269919e-01 2.20160925e-02]
 [9.87232858e-01 9.96826097e-01 1.00000000e+00]]
[[2.17362992e-01 1.32165748e-03 2.01542618e-04]
 [3.11710296e-01 5.20458083e-02 1.66910964e-01]
 [1.89300807e-01 1.65320444e-01 4.58571618e-07]
 ...
 [7.23135484e-01 3.40114804e-01 2.61404781e-01]
 [2.67474412e-01 3.51645671e-01 2.20160925e-02]
 [9.86847780e-01 9.96834060e-01 1.00000000e+00]]
[[2.17362992e-01 1.08064160e-03 3.05015150e-04]
 [3.11710296e-01 5.01979964e-02 1.74483059e-01]
 [1.89300807e-01 1.553

 /Users/muhammedsogut/Documents/GitHub/windfarm_modelling/GPy/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /Users/muhammedsogut/Documents/GitHub/windfarm_modelling/GPy/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


[[1.39870840e-02 1.04488162e-03 3.45413260e-04]
 [8.15770769e-02 5.65843051e-02 1.76857503e-01]
 [5.32366021e-03 1.94662862e-01 1.52322094e-06]
 ...
 [2.26415848e-01 3.43573352e-01 2.93487446e-01]
 [4.62594941e-02 4.02087312e-01 2.55753855e-02]
 [9.81261143e-01 9.97536359e-01 1.00000000e+00]]
[[1.39910810e-02 1.04486513e-03 3.45519748e-04]
 [8.15870617e-02 5.65783956e-02 1.76863448e-01]
 [5.32572122e-03 1.94626210e-01 1.52413799e-06]
 ...
 [2.26445573e-01 3.43553608e-01 2.93504475e-01]
 [4.62670911e-02 4.02040757e-01 2.55776093e-02]
 [9.81261474e-01 9.97535829e-01 1.00000000e+00]]
[[1.39924587e-02 1.04478297e-03 3.45613710e-04]
 [8.15905015e-02 5.65787726e-02 1.76868693e-01]
 [5.32643171e-03 1.94628883e-01 1.52492608e-06]
 ...
 [2.26455820e-01 3.43551158e-01 2.93519092e-01]
 [4.62697089e-02 4.02047974e-01 2.55795711e-02]
 [9.81261588e-01 9.97535928e-01 1.00000000e+00]]
[[1.39926300e-02 1.04466195e-03 3.45623518e-04]
 [8.15909306e-02 5.65752932e-02 1.76869241e-01]
 [5.32651999e-03 1.946

In [6]:
res.x

array([1.72065602, 2.54362242, 2.48427366, 1.12001978, 9.08207601,
       0.61244656])

In [7]:
res

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 0.29022266999396873
             x: [ 1.721e+00  2.544e+00  2.484e+00  1.120e+00  9.082e+00
                  6.124e-01]
           nit: 216
          nfev: 391
 final_simplex: (array([[ 1.721e+00,  2.544e+00, ...,  9.082e+00,
                         6.124e-01],
                       [ 1.721e+00,  2.544e+00, ...,  9.082e+00,
                         6.124e-01],
                       ...,
                       [ 1.721e+00,  2.544e+00, ...,  9.082e+00,
                         6.124e-01],
                       [ 1.721e+00,  2.544e+00, ...,  9.082e+00,
                         6.124e-01]]), array([ 2.902e-01,  2.902e-01,  2.902e-01,  2.902e-01,
                        2.902e-01,  2.902e-01,  2.902e-01]))

In [5]:
x0 = [4.25387599, 2.56450515, 2.33043463, 1.0151402, 8.04475192, 0.50753377]
bounds= ((0,10),(0,10),(0,10),(0,10),(0,10),(0,10))
res = differential_evolution(f,x0=x0, bounds= bounds, disp=True, polish=True)

differential_evolution step 1: f(x)= 0.284503
differential_evolution step 2: f(x)= 0.284503
differential_evolution step 3: f(x)= 0.284503
differential_evolution step 4: f(x)= 0.284503
differential_evolution step 5: f(x)= 0.284503


 /Users/muhammedsogut/Documents/GitHub/windfarm_modelling/GPy/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /Users/muhammedsogut/Documents/GitHub/windfarm_modelling/GPy/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


differential_evolution step 6: f(x)= 0.284503
differential_evolution step 7: f(x)= 0.284503
differential_evolution step 8: f(x)= 0.284503
differential_evolution step 9: f(x)= 0.284503
differential_evolution step 10: f(x)= 0.283604
differential_evolution step 11: f(x)= 0.281246
differential_evolution step 12: f(x)= 0.281246


 /Users/muhammedsogut/Documents/GitHub/windfarm_modelling/GPy/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


differential_evolution step 13: f(x)= 0.279075
differential_evolution step 14: f(x)= 0.279075


...

differential_evolution step 111: f(x)= 0.265156


 /Users/muhammedsogut/Documents/GitHub/windfarm_modelling/GPy/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/muhammedsogut/Documents/GitHub/windfarm_modelling/GPy/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /Users/muhammedsogut/Documents/GitHub/windfarm_modelling/GPy/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


differential_evolution step 112: f(x)= 0.265156


 /Users/muhammedsogut/Documents/GitHub/windfarm_modelling/GPy/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/muhammedsogut/Documents/GitHub/windfarm_modelling/GPy/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


differential_evolution step 113: f(x)= 0.265156
Polishing solution with 'L-BFGS-B'


In [6]:
res

 message: Optimization terminated successfully.
 success: True
     fun: 0.2650985017868992
       x: [ 1.749e+00  1.049e+00  8.825e+00  6.487e+00  1.934e+00
            6.024e+00]
     nit: 113
    nfev: 10456
     jac: [-2.005e-02  3.276e-02  3.170e-02  2.450e-02 -9.085e-03
            1.220e-02]

In [7]:
res.x

array([1.74856095, 1.04870804, 8.82515016, 6.4871528 , 1.93437115,
       6.02431617])